In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.draw import line

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from multiprocessing import Pool, cpu_count
from functools import partial

In [ ]:
import plotly.graph_objects as go

In [ ]:
import seaborn as sns

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
import geopandas as gpd

In [ ]:
df = pd.read_csv('metropoli_lat_lng.csv')
df.head()

#### 데이터 다운그레이드

In [ ]:
# 한글 폰트 설정 (Mac 기준 예시: AppleGothic, Windows는 'Malgun Gothic')
plt.rc('font', family='AppleGothic')  
# y축 숫자 줄임 없이 그대로 표시
plt.ticklabel_format(style='plain', axis='y')  

# 히스토그램 그리기
plt.hist(
    df['도착 행정동 주소'],
    histtype='bar'
)
plt.figure(figsize=(16, 8))
plt.xlabel('출발 행정동 주소')
plt.ylabel('빈도수')
plt.title('출발 행정동 주소 분포')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

##### 연도별 수도권 데이터 랜덤 추출 

In [ ]:
# 출발-도착 주소를 하나로 묶어 그룹화 기준 만들기
df['경로'] = df['출발 행정동 주소'] + " → " + df['도착 행정동 주소']

# 원하는 비율 지정
rates = [0.017, 0.10, 0.16]  

# 함수: 비율 유지한 샘플링
def stratified_sample(df, group_col, ratio):
    return (
        df.groupby(group_col, group_keys=False)
          .apply(lambda x: x.sample(frac=ratio, random_state=42))  # 재현 가능
    )

# 비율별로 샘플링
sample_1_7 = stratified_sample(df, '경로', 0.017)
sample_10 = stratified_sample(df, '경로', 0.10)
sample_16 = stratified_sample(df, '경로', 0.16)

In [ ]:
sample_1_7.columns

In [ ]:
sample_1_7.columns

In [ ]:
sample_1_7 = sample_1_7.drop(columns='경로')
sample_10 = sample_10.drop(columns='경로')
sample_16 = sample_16.drop(columns='경로')

In [ ]:
# 각 샘플 결측치 제거
sample_21= sample_1_7.dropna(subset=['출발 위도', '출발 경도', '도착 위도', '도착 경도'])
print(sample_21.shape)
sample_26= sample_10.dropna(subset=['출발 위도', '출발 경도', '도착 위도', '도착 경도'])
print(sample_26.shape)
sample_30= sample_16.dropna(subset=['출발 위도', '출발 경도', '도착 위도', '도착 경도'])
print(sample_30.shape)

In [ ]:
# 2. Kakao REST API 키
api_key = "4ffdaedd97065f6660c65fe7c75d42e6"  # 예: "69e904e85329cddd6795bf2cfe6c576a"

# 3. 주소 → 위도/경도 요청 함수
def get_lat_lng(address, api_key):
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        print(f"[Query] {address}")
        print(f"[Status] {response.status_code}")
        print(f"[JSON] {response.json()}")  # 응답 확인

        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                location = result['documents'][0]['address']
                return location['y'], location['x']
        return None, None
    except Exception as e:
        print(f"[Exception] {e}")
        return None, None

In [ ]:
#연도별 이동 데이터 파일로 저장 
sample_21.to_csv("sample_21.csv", index=False, encoding="utf-8-sig")
sample_26.to_csv("sample_26.csv", index=False, encoding="utf-8-sig")
sample_30.to_csv("sample_30.csv", index=False, encoding="utf-8-sig")

In [ ]:
sample_21

## 히트맵 그리기 (결측치 보완 X ver)

#### 히트맵 그리기

In [ ]:
samples = {
    "21": sample_21,
    "26": sample_26,
    "30": sample_30
}

# 결과 저장용 dict
heatmaps = {}
shapes = {}
grids = {}


In [ ]:
# 격자 크기 (약 300m)
lat_step = lon_step = 0.003

In [ ]:
# 히트맵 shape 계산
def h_w_shape(lat_max, lat_min, lat_step, lon_max, lon_min, lon_step):
    height = int((lat_max - lat_min) / lat_step) + 1
    width = int((lon_max - lon_min) / lon_step) + 1
    shape = (height, width)
    return height, width, shape

In [ ]:
#위경도를 그리드로 변환 함수
def latlon_to_grid(lat, lon, lat_min, lon_min, lat_step, lon_step):
    y = int((lat - lat_min) // lat_step)
    x = int((lon - lon_min) // lon_step)
    return x, y  # (열, 행)

In [ ]:
# 이동 경로 하나씩 그리면서 만개씩마다 디버깅 
def draw_lines_with_weight(shape, lines, k=5, verbose=True, log_every=10000):
    import numpy as np
    heatmap = np.zeros(shape, dtype=np.float32)

    for idx, (p1, p2) in enumerate(lines):
        x1, y1 = round(p1[0]), round(p1[1])
        x2, y2 = round(p2[0]), round(p2[1])

        rows, cols = line(y1, x1, y2, x2)

        for i in range(len(rows)):
            if 0 <= rows[i] < shape[0] and 0 <= cols[i] < shape[1]:
                heatmap[rows[i], cols[i]] += 3

        if 0 <= y1 < shape[0] and 0 <= x1 < shape[1]:
            heatmap[y1, x1] += (k - 1)
        if 0 <= y2 < shape[0] and 0 <= x2 < shape[1]:
            heatmap[y2, x2] += (k - 1)

        # 진행률 출력
        if verbose and idx % log_every == 0 and idx != 0:
            print(f"[{idx:,}/{len(lines):,}] 선 처리 중...")

    print(f"[완료] 총 {len(lines):,}개의 선 처리 완료.")
    return heatmap

In [ ]:
# 연도별 처리 루프
for year, df in samples.items():
    print(f"📦 {year}년 데이터 처리 중...")

    # 위경도 범위 계산
    lat_min = min(df['출발 위도'].min(), df['도착 위도'].min())
    lat_max = max(df['출발 위도'].max(), df['도착 위도'].max())
    lon_min = min(df['출발 경도'].min(), df['도착 경도'].min())
    lon_max = max(df['출발 경도'].max(), df['도착 경도'].max())

    # shape 계산
    height, width, shape = h_w_shape(lat_max, lat_min, lat_step, lon_max, lon_min, lon_step)
    shapes[year] = shape

    # 격자화한 라인 리스트 생성
    lines = []
    for _, row in df.iterrows():
        x1, y1 = latlon_to_grid(row['출발 위도'], row['출발 경도'], lat_min, lon_min, lat_step, lon_step)
        x2, y2 = latlon_to_grid(row['도착 위도'], row['도착 경도'], lat_min, lon_min, lat_step, lon_step)
        lines.append(((x1, y1), (x2, y2)))

    # 히트맵 생성
    heatmap = draw_lines_with_weight(shape, lines, k=5)
    heatmaps[year] = heatmap

    print(f"✅ {year}년 히트맵 shape: {shape}\n")

In [ ]:
for year, hm in heatmaps.items():  # heatmaps는 {연도: numpy array} 구조
    plt.figure(figsize=(12, 10))
    plt.imshow(hm, cmap="coolwarm", interpolation="nearest")
    plt.title(f"EV 이동 히트맵 - 20{year}년", fontsize=16)
    plt.colorbar(label="이동량")
    plt.axis("off")
    plt.tight_layout()
    plt.show()


## Plotly + Mapbox

#### 히트맵 기반 서울 plotly + mapbox 그리기

In [ ]:
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'browser'

lat_step = lon_step = 0.003
df_heats = {}

for year, heatmap in heatmaps.items():
    print(f"🗺️ Plotly 시각화: 20{year}년")

    shape = heatmap.shape
    height, width = shape

    # → 원래 위경도 최소값을 "샘플 데이터프레임"에서 계산
    df = samples[year]
    lat_min = min(df['출발 위도'].min(), df['도착 위도'].min())
    lon_min = min(df['출발 경도'].min(), df['도착 경도'].min())

    # 위경도 최대값은 shape로 유도
    lat_max = lat_min + lat_step * height
    lon_max = lon_min + lon_step * width

    # 히트맵에서 포인트 추출
    heat_points = []
    for y in range(height):
        for x in range(width):
            value = heatmap[y, x]
            if value > 0:
                lat = lat_min + y * lat_step + lat_step / 2
                lon = lon_min + x * lon_step + lon_step / 2
                heat_points.append([lat, lon, value])

    df_heat = pd.DataFrame(heat_points, columns=["lat", "lon", "value"])
    df_heats[year] = df_heat
    
    # Plotly density_mapbox 시각화
    fig = px.density_mapbox(
        df_heat,
        lat="lat",
        lon="lon",
        z="value",
        radius=1,
        center=dict(lat=(lat_min + lat_max) / 2, lon=(lon_min + lon_max) / 2),
        zoom=10,
        mapbox_style="carto-positron",
        color_continuous_scale="plasma",
        title=f"EV 이동 히트맵 - 20{year}년"
    )

    fig.show()
    fig.write_html(f"ev_heatmap_plotly_{year}.html")

In [ ]:
df_heats["30"].max()

In [ ]:
fig.write_html("ev_heatmap_plotly.html")

#### 이동 불가 지역 히트 포인트 제거

In [ ]:
df_heats

###### 이동 불가 제한 지역은 shp 파일을 QGIS에서 geojson으로 변경

In [ ]:
# 이동 제한 구역 geojson 파일 불러오기
restricted = gpd.read_file("merged_restricted_areas.geojson")

In [ ]:
restricted = restricted.drop(columns = ['path'])
restricted = restricted.to_crs("EPSG:4326")
restricted.head()

In [ ]:
# 이동 불가 지역 시각화 
restricted.plot(figsize=(10, 10), color="lightblue", edgecolor="black")
plt.title("제한 지역 폴리곤 (하천, 산지, 녹지 등)")
plt.axis("off")
plt.show()

In [ ]:
from shapely.geometry import Point

df_heats_filtered = {}

#이동 불가 지역 폴리곤 안에 히트맵 포인트가 있는지 확인하고 포함되지 않은 히트맵 포인트만 필터링하는 함수 
for year, df in df_heats.items():
    print(f"📦 {year}년 필터링 중...")

    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs="EPSG:4326"
    )

    joined = gpd.sjoin(gdf, restricted, how="left", predicate="within")

    # restricted에 포함되지 않은 것만 필터링
    gdf_filtered = joined[joined.index_right.isna()].copy()

    # 오른쪽 restricted 속성 제거 (선택)
    gdf_filtered.drop(columns=[col for col in gdf_filtered.columns if col.endswith('_right') or col in restricted.columns], inplace=True)

    df_heats_filtered[year] = gdf_filtered
    print(f"✅ {year}년 → {len(df)}개 중 {len(gdf_filtered)}개 유지")

In [ ]:
#이동 불가 지역 반영 연도별 데이터 프레임 
for year, df in df_heats_filtered.items():
    df.to_csv(f"{year}년도_수도권_이동_필터링.csv")

## 수도권 공급 + 수요 지도 합치기 

필요한 데이터 
- 공급: 수도권 전기차 충전소 위치
- 수요: 수도권 이동 동선 히트맵 2021, 2026, 2030년
- 공영 주차장 

In [ ]:
# 공급 데이터 로드 
s = pd.read_csv("서울_충전소_좌표추가_전체_cleaned.csv")
g = pd.read_csv("경기도_충전소_좌표추가_전체_cleaned.csv")
i = pd.read_csv("인천광역시_충전소_좌표추가_전체_cleaned.csv")

print(s.columns)
print(g.columns)
print(i.columns)

#서울 충전소명, 충전기ID, 위도, 경도
#경기도 충전소명, 충전기ID, 위도, 경도

chargers = pd.concat([s[['충전소명', '충전기ID', '위도', '경도']], g[['충전소명', '충전기ID', '위도', '경도']], i[['충전소명', '충전기ID', '위도', '경도']]])
print(chargers.shape)
# 수요 데이터는
# df_heats

In [ ]:
# 공영 주차장 로드
pk = pd.read_csv("수도권_공영주차장데이터.csv")
pk.head()

In [ ]:
pk = pk[['주차장명', '위도', '경도']]
pk = pk.rename(columns={"위도": "lat", "경도": "lon"})

In [ ]:
pk.to_csv("수도권_공영주차장.csv", index = False, encoding="utf-8-sig")

#### 21년도 처리 

In [ ]:
#21년도 수요
d_21 = df_heats_filtered['21'].copy()
d_21.columns

#21년도 수요 
d_21.rename(
    columns = {
        "value" : "이동경로량"
},inplace = True
)

In [ ]:
#21년도 공급
chargers.rename(
    columns = {
        "위도" : "lat",
        "경도" : "lon"
    }, inplace = True
)

print(d_21.head())
print(chargers.head())

In [ ]:
# 행정동 별 충전소 개수 + 충전기 개수 모두 계산하는 함수
def count_chargers_and_units_nearby(row, chargers, radius=0.003):
    mask = ((abs(chargers["lat"] - row["lat"]) < radius) &
            (abs(chargers["lon"] - row["lon"]) < radius))
    
    nearby = chargers[mask]
    
    station_count = nearby["충전소명"].nunique()  # 고유 충전소 수
    unit_count = nearby["충전기ID"].count()      # 충전기 개수
    
    return pd.Series([station_count, unit_count])]

d_21[["charger_station_count", "charger_total_count"]] = d_21.apply(
    lambda row: count_chargers_and_units_nearby(row, chargers),
    axis=1
)

In [ ]:
d_21.head()

In [ ]:
# 수요-공급 차이 계산
d_21["gap"] = d_21["이동경로량"] - d_21["charger_count"]

#수요- 공급 차이 중 유의미한 차이 두두러지게 스케일링: signed logrithimic 
def signed_log(x):
    return np.sign(x) * np.log1p(abs(x))

d_21["gap_signedlog"] = d_21["gap"].apply(signed_log)

In [ ]:
# Signed Log Gap 분포 확인 
sns.histplot(
    data=d_21,
    x="gap_signedlog",
    bins=100,
    kde=True,
    stat="count",
    label="Signed Log Gap",
    color="tomato",
    alpha=0.6
)

# 📍 마무리 설정
plt.xlabel("Gap")
plt.ylabel("num")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
d_21.to_csv("21년도_수요-이동량_스케일링.csv",index=False, encoding="utf-8-sig")

In [ ]:
d_21["gap_signedlog_emph"] = d_21["gap_signedlog"].apply(lambda x: np.sign(x) * (abs(x) ** 1.5))
d_21["gap_size"] = d_21["gap_signedlog_emph"].abs()

# 상위 10%, 하위 10% 추출
upper_10_21 = d_21[d_21["gap_signedlog_emph"] >= d_21["gap_signedlog_emph"].quantile(0.9)]
lower_10_21 = d_21[d_21["gap_signedlog_emph"] <= d_21["gap_signedlog_emph"].quantile(0.1)]
filtered_21 = pd.concat([upper_10_21, lower_10_21])

# 지도 시각화
fig = px.scatter_mapbox(
    filtered_21,
    lat="lat",
    lon="lon",
    color="gap_signedlog_emph",
    color_continuous_scale="RdBu",
    color_continuous_midpoint=0,
    size_max=20,
    zoom=10,
    hover_data={
        "gap_signedlog_emph": True,
        "이동경로량": True,
        "charger_count": True,
        "lat": False,
        "lon": False
    },
    title="EV 격차 상/하위 10% + 충전소 + 공영주차장 (2021년)"
)

# 공영주차장 핀 추가
fig.add_trace(go.Scattermapbox(
    lat=pk["lat"],
    lon=pk["lon"],
    mode="markers",
    marker=go.scattermapbox.Marker(size=3, color="yellow"),
    text=pk["주차장명"],
    hoverinfo="text",
    name="공영주차장"
))

# 지도 스타일 설정 및 저장
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=10
)

fig.write_html("격차_상하위10+충전소+공영주차장_2021.html")
fig.show()



#### 26년 처리

In [ ]:
#21년도 수요
d_26 = df_heats_filtered['26'].copy()
d_26.columns

#21년도 수요 
d_26.rename(
    columns = {
        "value" : "이동경로량"
},inplace = True
)

In [ ]:
d_26[["charger_station_count", "charger_total_count"]] = d_26.apply(
    lambda row: count_chargers_and_units_nearby(row, chargers),
    axis=1
)

In [ ]:
# # 행정구역별 충전소 수
# d_26["charger_count"] = d_26.apply(
#     lambda row: count_chargers_nearby(row, chargers), axis=1
# )


In [ ]:
# 수요-공급 차이 계산
d_26["gap"] = d_26["이동경로량"] - d_26["charger_count"]
d_26.head()

In [ ]:
# 수요-공급 차이 계산
d_26["gap"] = d_26["이동경로량"] - d_26["charger_count"]

#signed logrithimic scaling 
def signed_log(x):
    return np.sign(x) * np.log1p(abs(x))

d_26["gap_signedlog"] = d_26["gap"].apply(signed_log)

In [ ]:
d_26.head()

In [ ]:
d_26.to_csv("26년도_수요-이동량_스케일링.csv",index=False, encoding="utf-8-sig")

In [ ]:
d_26["gap_signedlog_emph"] = d_26["gap_signedlog"].apply(lambda x: np.sign(x) * (abs(x) ** 1.5))
d_26["gap_size"] = d_26["gap_signedlog_emph"].abs()

# 상위 10%, 하위 10% 추출
upper_10_26 = d_26[d_26["gap_signedlog_emph"] >= d_26["gap_signedlog_emph"].quantile(0.9)]
lower_10_26 = d_26[d_26["gap_signedlog_emph"] <= d_26["gap_signedlog_emph"].quantile(0.1)]
filtered_26 = pd.concat([upper_10_26, lower_10_26])

# 지도 시각화
fig = px.scatter_mapbox(
    filtered_26,
    lat="lat",
    lon="lon",
    color="gap_signedlog_emph",
    color_continuous_scale="PrGN",
    color_continuous_midpoint=0,
    size_max=20,
    zoom=10,
    hover_data={
        "gap_signedlog_emph": True,
        "이동경로량": True,
        "charger_count": True,
        "lat": False,
        "lon": False
    },
    title="EV 격차 상/하위 10% + 충전소 + 공영주차장 (2021년)"
)

# 공영주차장 핀 추가
fig.add_trace(go.Scattermapbox(
    lat=pk["lat"],
    lon=pk["lon"],
    mode="markers",
    marker=go.scattermapbox.Marker(size=3, color="yellow"),
    text=pk["주차장명"],
    hoverinfo="text",
    name="공영주차장"
))

# 지도 스타일 설정 및 저장
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=10
)

fig.write_html("격차_상하위10+충전소+공영주차장_2026.html")
fig.show()



#### 30년 처리

In [ ]:
#21년도 수요
d_30 = df_heats_filtered['30'].copy()
d_30.columns

#21년도 수요 
d_30.rename(
    columns = {
        "value" : "이동경로량"
},inplace = True
)

In [ ]:
# 행정구역별 충전소 수
d_30["charger_count"] = d_30.apply(
    lambda row: count_chargers_nearby(row, chargers), axis=1
)

In [ ]:
# 수요-공급 차이 계산
d_30["gap"] = d_30["이동경로량"] - d_30["charger_count"]

#signed logrithimic scaling 
def signed_log(x):
    return np.sign(x) * np.log1p(abs(x))

d_30["gap_signedlog"] = d_30["gap"].apply(signed_log)

In [ ]:
d_30.to_csv("30년도_수요-이동량_스케일링.csv",index=False, encoding="utf-8-sig")

In [ ]:
d_30["gap_signedlog_emph"] = d_30["gap_signedlog"].apply(lambda x: np.sign(x) * (abs(x) ** 1.5))
d_30["gap_size"] = d_30["gap_signedlog_emph"].abs()

# 상위 10%, 하위 10% 추출
upper_10_30 = d_30[d_30["gap_signedlog_emph"] >= d_30["gap_signedlog_emph"].quantile(0.9)]
lower_10_30 = d_30[d_30["gap_signedlog_emph"] <= d_30["gap_signedlog_emph"].quantile(0.1)]
filtered_30 = pd.concat([upper_10_30, lower_10_30])

# 지도 시각화
fig = px.scatter_mapbox(
    filtered_30,
    lat="lat",
    lon="lon",
    color="gap_signedlog_emph",
    color_continuous_scale="PrGN",
    color_continuous_midpoint=0,
    size_max=20,
    zoom=10,
    hover_data={
        "gap_signedlog_emph": True,
        "이동경로량": True,
        "charger_count": True,
        "lat": False,
        "lon": False
    },
    title="EV 격차 상/하위 10% + 충전소 + 공영주차장 (2021년)"
)

# 공영주차장 핀 추가
fig.add_trace(go.Scattermapbox(
    lat=pk["lat"],
    lon=pk["lon"],
    mode="markers",
    marker=go.scattermapbox.Marker(size=3, color="yellow"),
    text=pk["주차장명"],
    hoverinfo="text",
    name="공영주차장"
))

# 지도 스타일 설정 및 저장
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=10
)

fig.write_html("격차_상하위10+충전소+공영주차장_2030.html")
fig.show()

